Scoring in next 10 actions model - Model Tuning and Building - GBM

In [2]:
import sys
sys.path.append("/Users/ciaran/Documents/Projects/AFL/git-repositories/afl-player-ratings/")

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import joblib
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

from vaep.config import vaep_features_file_path, vaep_labels_file_path
# from vaep.domain.preprocessing.dataessor import DataPreprocessor
# from vaep.domain.contracts.mappings import Mappings
from vaep.domain.contracts.modelling_data_contract import ModellingDataContract
from vaep.domain.modelling.hyperparameter_tuning import XGBHyperparameterTuner
from vaep.domain.modelling.supermodel import SuperXGBClassifier
from vaep.domain.modelling.optuna_xgb_param_grid import OptunaXGBParamGrid

pd.options.display.max_rows = 100
pd.options.display.max_columns = 999

%load_ext autoreload
%autoreload 2

Model Version

In [4]:
model_version = 1
model_name = 'vaep_scoring'
model_file_name = model_name + '_v' + str(model_version)

model_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/afl-player-ratings/vaep/model_outputs/models"
prediction_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/afl-player-ratings/vaep/model_outputs/predictions"

preprocessor_file_name = 'preprocessor_v' + str(model_version)
preprocessor_output_path = ""

In [5]:
RESPONSE = ModellingDataContract.RESPONSE_SCORES

In [6]:
# FEATURES = ModellingDataContract.open_goal_modelling_feature_list

In [7]:
# MONOTONE_CONSTRAINTS = ModellingDataContract.monotone_constraints_open_goal

Load Data

In [8]:
df_modelling = pd.read_csv(vaep_features_file_path)
df_modelling.tail()

,action_type,type_Kick,type_Handball,type_Carry,type_Uncontested Mark,type_Contested Mark,type_Loose Ball Get,type_Hard Ball Get,type_Spoil,type_Gather,type_Free For,type_Knock On,type_Shot,type_Tackle,type_Error,outcome_type,outcome_effective,outcome_ineffective,outcome_clanger,type_Kick_outcome_effective,type_Kick_outcome_ineffective,type_Kick_outcome_clanger,type_Handball_outcome_effective,type_Handball_outcome_ineffective,type_Handball_outcome_clanger,type_Carry_outcome_effective,type_Carry_outcome_ineffective,type_Carry_outcome_clanger,type_Uncontested Mark_outcome_effective,type_Uncontested Mark_outcome_ineffective,type_Uncontested Mark_outcome_clanger,type_Contested Mark_outcome_effective,type_Contested Mark_outcome_ineffective,type_Contested Mark_outcome_clanger,type_Loose Ball Get_outcome_effective,type_Loose Ball Get_outcome_ineffective,type_Loose Ball Get_outcome_clanger,type_Hard Ball Get_outcome_effective,type_Hard Ball Get_outcome_ineffective,type_Hard Ball Get_outcome_clanger,type_Spoil_outcome_effective,type_Spoil_outcome_ineffective,type_Spoil_outcome_clanger,type_Gather_outcome_effective,type_Gather_outcome_ineffective,type_Gather_outcome_clanger,type_Free For_outcome_effective,type_Free For_outcome_ineffective,type_Free For_outcome_clanger,type_Knock On_outcome_effective,type_Knock On_outcome_ineffective,type_Knock On_outcome_clanger,type_Shot_outcome_effective,type_Shot_outcome_ineffective,type_Shot_outcome_clanger,type_Tackle_outcome_effective,type_Tackle_outcome_ineffective,type_Tackle_outcome_clanger,type_Error_outcome_effective,type_Error_outcome_ineffective,type_Error_outcome_clanger,quarter,quarter_seconds,overall_seconds,start_x,start_y,end_x,end_y,dx,dy,movement,action_type.1,type_Kick.1,type_Handball.1,type_Carry.1,type_Uncontested Mark.1,type_Contested Mark.1,type_Loose Ball Get.1,type_Hard Ball Get.1,type_Spoil.1,type_Gather.1,type_Free For.1,type_Knock On.1,type_Shot.1,type_Tackle.1,type_Error.1,outcome_type.1,outcome_effective.1,outcome_ineffective.1,outcome_clanger.1,type_Kick_outcome_effective.1,type_Kick_outcome_ineffective.1,type_Kick_outcome_clanger.1,type_Handball_outcome_effective.1,type_Handball_outcome_ineffective.1,type_Handball_outcome_clanger.1,type_Carry_outcome_effective.1,type_Carry_outcome_ineffective.1,type_Carry_outcome_clanger.1,type_Uncontested Mark_outcome_effective.1,type_Uncontested Mark_outcome_ineffective.1,type_Uncontested Mark_outcome_clanger.1,type_Contested Mark_outcome_effective.1,type_Contested Mark_outcome_ineffective.1,type_Contested Mark_outcome_clanger.1,type_Loose Ball Get_outcome_effective.1,type_Loose Ball Get_outcome_ineffective.1,type_Loose Ball Get_outcome_clanger.1,type_Hard Ball Get_outcome_effective.1,type_Hard Ball Get_outcome_ineffective.1,type_Hard Ball Get_outcome_clanger.1,type_Spoil_outcome_effective.1,type_Spoil_outcome_ineffective.1,type_Spoil_outcome_clanger.1,type_Gather_outcome_effective.1,type_Gather_outcome_ineffective.1,type_Gather_outcome_clanger.1,type_Free For_outcome_effective.1,type_Free For_outcome_ineffective.1,type_Free For_outcome_clanger.1,type_Knock On_outcome_effective.1,type_Knock On_outcome_ineffective.1,type_Knock On_outcome_clanger.1,type_Shot_outcome_effective.1,type_Shot_outcome_ineffective.1,type_Shot_outcome_clanger.1,type_Tackle_outcome_effective.1,type_Tackle_outcome_ineffective.1,type_Tackle_outcome_clanger.1,type_Error_outcome_effective.1,type_Error_outcome_ineffective.1,type_Error_outcome_clanger.1,quarter.1,quarter_seconds.1,overall_seconds.1,start_x.1,start_y.1,end_x.1,end_y.1,dx.1,dy.1,movement.1,action_type.2,type_Kick.2,type_Handball.2,type_Carry.2,type_Uncontested Mark.2,type_Contested Mark.2,type_Loose Ball Get.2,type_Hard Ball Get.2,type_Spoil.2,type_Gather.2,type_Free For.2,type_Knock On.2,type_Shot.2,type_Tackle.2,type_Error.2,outcome_type.2,outcome_effective.2,outcome_ineffective.2,outcome_clanger.2,type_Kick_outcome_effective.2,type_Kick_outcome_ineffective.2,type_Kick_outcome_clanger.2,type_Handball_

In [9]:
list(df_modelling)

['action_type',
 'type_Kick',
 'type_Handball',
 'type_Carry',
 'type_Uncontested Mark',
 'type_Contested Mark',
 'type_Loose Ball Get',
 'type_Hard Ball Get',
 'type_Spoil',
 'type_Gather',
 'type_Free For',
 'type_Knock On',
 'type_Shot',
 'type_Tackle',
 'type_Error',
 'outcome_type',
 'outcome_effective',
 'outcome_ineffective',
 'outcome_clanger',
 'type_Kick_outcome_effective',
 'type_Kick_outcome_ineffective',
 'type_Kick_outcome_clanger',
 'type_Handball_outcome_effective',
 'type_Handball_outcome_ineffective',
 'type_Handball_outcome_clanger',
 'type_Carry_outcome_effective',
 'type_Carry_outcome_ineffective',
 'type_Carry_outcome_clanger',
 'type_Uncontested Mark_outcome_effective',
 'type_Uncontested Mark_outcome_ineffective',
 'type_Uncontested Mark_outcome_clanger',
 'type_Contested Mark_outcome_effective',
 'type_Contested Mark_outcome_ineffective',
 'type_Contested Mark_outcome_clanger',
 'type_Loose Ball Get_outcome_effective',
 'type_Loose Ball Get_outcome_ineffective'

In [10]:
labels = pd.read_csv(vaep_labels_file_path)
labels = labels[RESPONSE]

Train Test Split

In [11]:
X, y = df_modelling, labels
X_modelling, X_test, y_modelling, y_test = train_test_split(X, y, test_size = 0.2, random_state=2407, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_modelling, y_modelling, test_size = 0.2, random_state=2407, stratify=y_modelling)


In [12]:
y_train.mean(), y_test.mean(), y_val.mean()

(0.11534220065375746, 0.11534038759771703, 0.11534201052340072)

In [13]:
X_test.head()

,action_type,type_Kick,type_Handball,type_Carry,type_Uncontested Mark,type_Contested Mark,type_Loose Ball Get,type_Hard Ball Get,type_Spoil,type_Gather,type_Free For,type_Knock On,type_Shot,type_Tackle,type_Error,outcome_type,outcome_effective,outcome_ineffective,outcome_clanger,type_Kick_outcome_effective,type_Kick_outcome_ineffective,type_Kick_outcome_clanger,type_Handball_outcome_effective,type_Handball_outcome_ineffective,type_Handball_outcome_clanger,type_Carry_outcome_effective,type_Carry_outcome_ineffective,type_Carry_outcome_clanger,type_Uncontested Mark_outcome_effective,type_Uncontested Mark_outcome_ineffective,type_Uncontested Mark_outcome_clanger,type_Contested Mark_outcome_effective,type_Contested Mark_outcome_ineffective,type_Contested Mark_outcome_clanger,type_Loose Ball Get_outcome_effective,type_Loose Ball Get_outcome_ineffective,type_Loose Ball Get_outcome_clanger,type_Hard Ball Get_outcome_effective,type_Hard Ball Get_outcome_ineffective,type_Hard Ball Get_outcome_clanger,type_Spoil_outcome_effective,type_Spoil_outcome_ineffective,type_Spoil_outcome_clanger,type_Gather_outcome_effective,type_Gather_outcome_ineffective,type_Gather_outcome_clanger,type_Free For_outcome_effective,type_Free For_outcome_ineffective,type_Free For_outcome_clanger,type_Knock On_outcome_effective,type_Knock On_outcome_ineffective,type_Knock On_outcome_clanger,type_Shot_outcome_effective,type_Shot_outcome_ineffective,type_Shot_outcome_clanger,type_Tackle_outcome_effective,type_Tackle_outcome_ineffective,type_Tackle_outcome_clanger,type_Error_outcome_effective,type_Error_outcome_ineffective,type_Error_outcome_clanger,quarter,quarter_seconds,overall_seconds,start_x,start_y,end_x,end_y,dx,dy,movement,action_type.1,type_Kick.1,type_Handball.1,type_Carry.1,type_Uncontested Mark.1,type_Contested Mark.1,type_Loose Ball Get.1,type_Hard Ball Get.1,type_Spoil.1,type_Gather.1,type_Free For.1,type_Knock On.1,type_Shot.1,type_Tackle.1,type_Error.1,outcome_type.1,outcome_effective.1,outcome_ineffective.1,outcome_clanger.1,type_Kick_outcome_effective.1,type_Kick_outcome_ineffective.1,type_Kick_outcome_clanger.1,type_Handball_outcome_effective.1,type_Handball_outcome_ineffective.1,type_Handball_outcome_clanger.1,type_Carry_outcome_effective.1,type_Carry_outcome_ineffective.1,type_Carry_outcome_clanger.1,type_Uncontested Mark_outcome_effective.1,type_Uncontested Mark_outcome_ineffective.1,type_Uncontested Mark_outcome_clanger.1,type_Contested Mark_outcome_effective.1,type_Contested Mark_outcome_ineffective.1,type_Contested Mark_outcome_clanger.1,type_Loose Ball Get_outcome_effective.1,type_Loose Ball Get_outcome_ineffective.1,type_Loose Ball Get_outcome_clanger.1,type_Hard Ball Get_outcome_effective.1,type_Hard Ball Get_outcome_ineffective.1,type_Hard Ball Get_outcome_clanger.1,type_Spoil_outcome_effective.1,type_Spoil_outcome_ineffective.1,type_Spoil_outcome_clanger.1,type_Gather_outcome_effective.1,type_Gather_outcome_ineffective.1,type_Gather_outcome_clanger.1,type_Free For_outcome_effective.1,type_Free For_outcome_ineffective.1,type_Free For_outcome_clanger.1,type_Knock On_outcome_effective.1,type_Knock On_outcome_ineffective.1,type_Knock On_outcome_clanger.1,type_Shot_outcome_effective.1,type_Shot_outcome_ineffective.1,type_Shot_outcome_clanger.1,type_Tackle_outcome_effective.1,type_Tackle_outcome_ineffective.1,type_Tackle_outcome_clanger.1,type_Error_outcome_effective.1,type_Error_outcome_ineffective.1,type_Error_outcome_clanger.1,quarter.1,quarter_seconds.1,overall_seconds.1,start_x.1,start_y.1,end_x.1,end_y.1,dx.1,dy.1,movement.1,action_type.2,type_Kick.2,type_Handball.2,type_Carry.2,type_Uncontested Mark.2,type_Contested Mark.2,type_Loose Ball Get.2,type_Hard Ball Get.2,type_Spoil.2,type_Gather.2,type_Free For.2,type_Knock On.2,type_Shot.2,type_Tackle.2,type_Error.2,outcome_type.2,outcome_effective.2,outcome_ineffective.2,outcome_clanger.2,type_Kick_outcome_effective.2,type_Kick_outcome_ineffective.2,type_Kick_outcome_clanger.2,type_Handball_

Preprocess Data

In [14]:
# preprocessor = DataPreprocessor()
# preprocessor.fit(X_train)

# X_train = preprocessor.transform(X_train)
# X_test = preprocessor.transform(X_test)

In [15]:
categorical_cols = set(list(X_train)) - set(list(X_train.select_dtypes('number')) + list(X_train.select_dtypes('bool')))

In [16]:
X_train_preproc = X_train.drop(columns = categorical_cols)
X_val_preproc = X_val.drop(columns = categorical_cols)
X_test_preproc = X_test.drop(columns = categorical_cols)

In [17]:
X_train_preproc.shape, X_val_preproc.shape, X_test_preproc.shape

((606647, 220), (151662, 220), (189578, 220))

In [18]:
list(X_train_preproc)

['type_Kick',
 'type_Handball',
 'type_Carry',
 'type_Uncontested Mark',
 'type_Contested Mark',
 'type_Loose Ball Get',
 'type_Hard Ball Get',
 'type_Spoil',
 'type_Gather',
 'type_Free For',
 'type_Knock On',
 'type_Shot',
 'type_Tackle',
 'type_Error',
 'outcome_effective',
 'outcome_ineffective',
 'outcome_clanger',
 'type_Kick_outcome_effective',
 'type_Kick_outcome_ineffective',
 'type_Kick_outcome_clanger',
 'type_Handball_outcome_effective',
 'type_Handball_outcome_ineffective',
 'type_Handball_outcome_clanger',
 'type_Carry_outcome_effective',
 'type_Carry_outcome_ineffective',
 'type_Carry_outcome_clanger',
 'type_Uncontested Mark_outcome_effective',
 'type_Uncontested Mark_outcome_ineffective',
 'type_Uncontested Mark_outcome_clanger',
 'type_Contested Mark_outcome_effective',
 'type_Contested Mark_outcome_ineffective',
 'type_Contested Mark_outcome_clanger',
 'type_Loose Ball Get_outcome_effective',
 'type_Loose Ball Get_outcome_ineffective',
 'type_Loose Ball Get_outcome_c

In [38]:
X_train_preproc.head()

,type_Kick,type_Handball,type_Carry,type_Uncontested Mark,type_Contested Mark,type_Loose Ball Get,type_Hard Ball Get,type_Spoil,type_Gather,type_Free For,type_Knock On,type_Shot,type_Tackle,type_Error,outcome_effective,outcome_ineffective,outcome_clanger,type_Kick_outcome_effective,type_Kick_outcome_ineffective,type_Kick_outcome_clanger,type_Handball_outcome_effective,type_Handball_outcome_ineffective,type_Handball_outcome_clanger,type_Carry_outcome_effective,type_Carry_outcome_ineffective,type_Carry_outcome_clanger,type_Uncontested Mark_outcome_effective,type_Uncontested Mark_outcome_ineffective,type_Uncontested Mark_outcome_clanger,type_Contested Mark_outcome_effective,type_Contested Mark_outcome_ineffective,type_Contested Mark_outcome_clanger,type_Loose Ball Get_outcome_effective,type_Loose Ball Get_outcome_ineffective,type_Loose Ball Get_outcome_clanger,type_Hard Ball Get_outcome_effective,type_Hard Ball Get_outcome_ineffective,type_Hard Ball Get_outcome_clanger,type_Spoil_outcome_effective,type_Spoil_outcome_ineffective,type_Spoil_outcome_clanger,type_Gather_outcome_effective,type_Gather_outcome_ineffective,type_Gather_outcome_clanger,type_Free For_outcome_effective,type_Free For_outcome_ineffective,type_Free For_outcome_clanger,type_Knock On_outcome_effective,type_Knock On_outcome_ineffective,type_Knock On_outcome_clanger,type_Shot_outcome_effective,type_Shot_outcome_ineffective,type_Shot_outcome_clanger,type_Tackle_outcome_effective,type_Tackle_outcome_ineffective,type_Tackle_outcome_clanger,type_Error_outcome_effective,type_Error_outcome_ineffective,type_Error_outcome_clanger,quarter,quarter_seconds,overall_seconds,start_x,start_y,end_x,end_y,dx,dy,movement,type_Kick.1,type_Handball.1,type_Carry.1,type_Uncontested Mark.1,type_Contested Mark.1,type_Loose Ball Get.1,type_Hard Ball Get.1,type_Spoil.1,type_Gather.1,type_Free For.1,type_Knock On.1,type_Shot.1,type_Tackle.1,type_Error.1,outcome_effective.1,outcome_ineffective.1,outcome_clanger.1,type_Kick_outcome_effective.1,type_Kick_outcome_ineffective.1,type_Kick_outcome_clanger.1,type_Handball_outcome_effective.1,type_Handball_outcome_ineffective.1,type_Handball_outcome_clanger.1,type_Carry_outcome_effective.1,type_Carry_outcome_ineffective.1,type_Carry_outcome_clanger.1,type_Uncontested Mark_outcome_effective.1,type_Uncontested Mark_outcome_ineffective.1,type_Uncontested Mark_outcome_clanger.1,type_Contested Mark_outcome_effective.1,type_Contested Mark_outcome_ineffective.1,type_Contested Mark_outcome_clanger.1,type_Loose Ball Get_outcome_effective.1,type_Loose Ball Get_outcome_ineffective.1,type_Loose Ball Get_outcome_clanger.1,type_Hard Ball Get_outcome_effective.1,type_Hard Ball Get_outcome_ineffective.1,type_Hard Ball Get_outcome_clanger.1,type_Spoil_outcome_effective.1,type_Spoil_outcome_ineffective.1,type_Spoil_outcome_clanger.1,type_Gather_outcome_effective.1,type_Gather_outcome_ineffective.1,type_Gather_outcome_clanger.1,type_Free For_outcome_effective.1,type_Free For_outcome_ineffective.1,type_Free For_outcome_clanger.1,type_Knock On_outcome_effective.1,type_Knock On_outcome_ineffective.1,type_Knock On_outcome_clanger.1,type_Shot_outcome_effective.1,type_Shot_outcome_ineffective.1,type_Shot_outcome_clanger.1,type_Tackle_outcome_effective.1,type_Tackle_outcome_ineffective.1,type_Tackle_outcome_clanger.1,type_Error_outcome_effective.1,type_Error_outcome_ineffective.1,type_Error_outcome_clanger.1,quarter.1,quarter_seconds.1,overall_seconds.1,start_x.1,start_y.1,end_x.1,end_y.1,dx.1,dy.1,movement.1,type_Kick.2,type_Handball.2,type_Carry.2,type_Uncontested Mark.2,type_Contested Mark.2,type_Loose Ball Get.2,type_Hard Ball Get.2,type_Spoil.2,type_Gather.2,type_Free For.2,type_Knock On.2,type_Shot.2,type_Tackle.2,type_Error.2,outcome_effective.2,outcome_ineffective.2,outcome_clanger.2,type_Kick_outcome_effective.2,type_Kick_outcome_ineffective.2,type_Kick_outcome_clanger.2,type_Handball_outcome_effective.2,type_Handball_outcome_ineffective.2,type_Handball_outcome_clang

In [39]:
X_test_preproc.head()

,type_Kick,type_Handball,type_Carry,type_Uncontested Mark,type_Contested Mark,type_Loose Ball Get,type_Hard Ball Get,type_Spoil,type_Gather,type_Free For,type_Knock On,type_Shot,type_Tackle,type_Error,outcome_effective,outcome_ineffective,outcome_clanger,type_Kick_outcome_effective,type_Kick_outcome_ineffective,type_Kick_outcome_clanger,type_Handball_outcome_effective,type_Handball_outcome_ineffective,type_Handball_outcome_clanger,type_Carry_outcome_effective,type_Carry_outcome_ineffective,type_Carry_outcome_clanger,type_Uncontested Mark_outcome_effective,type_Uncontested Mark_outcome_ineffective,type_Uncontested Mark_outcome_clanger,type_Contested Mark_outcome_effective,type_Contested Mark_outcome_ineffective,type_Contested Mark_outcome_clanger,type_Loose Ball Get_outcome_effective,type_Loose Ball Get_outcome_ineffective,type_Loose Ball Get_outcome_clanger,type_Hard Ball Get_outcome_effective,type_Hard Ball Get_outcome_ineffective,type_Hard Ball Get_outcome_clanger,type_Spoil_outcome_effective,type_Spoil_outcome_ineffective,type_Spoil_outcome_clanger,type_Gather_outcome_effective,type_Gather_outcome_ineffective,type_Gather_outcome_clanger,type_Free For_outcome_effective,type_Free For_outcome_ineffective,type_Free For_outcome_clanger,type_Knock On_outcome_effective,type_Knock On_outcome_ineffective,type_Knock On_outcome_clanger,type_Shot_outcome_effective,type_Shot_outcome_ineffective,type_Shot_outcome_clanger,type_Tackle_outcome_effective,type_Tackle_outcome_ineffective,type_Tackle_outcome_clanger,type_Error_outcome_effective,type_Error_outcome_ineffective,type_Error_outcome_clanger,quarter,quarter_seconds,overall_seconds,start_x,start_y,end_x,end_y,dx,dy,movement,type_Kick.1,type_Handball.1,type_Carry.1,type_Uncontested Mark.1,type_Contested Mark.1,type_Loose Ball Get.1,type_Hard Ball Get.1,type_Spoil.1,type_Gather.1,type_Free For.1,type_Knock On.1,type_Shot.1,type_Tackle.1,type_Error.1,outcome_effective.1,outcome_ineffective.1,outcome_clanger.1,type_Kick_outcome_effective.1,type_Kick_outcome_ineffective.1,type_Kick_outcome_clanger.1,type_Handball_outcome_effective.1,type_Handball_outcome_ineffective.1,type_Handball_outcome_clanger.1,type_Carry_outcome_effective.1,type_Carry_outcome_ineffective.1,type_Carry_outcome_clanger.1,type_Uncontested Mark_outcome_effective.1,type_Uncontested Mark_outcome_ineffective.1,type_Uncontested Mark_outcome_clanger.1,type_Contested Mark_outcome_effective.1,type_Contested Mark_outcome_ineffective.1,type_Contested Mark_outcome_clanger.1,type_Loose Ball Get_outcome_effective.1,type_Loose Ball Get_outcome_ineffective.1,type_Loose Ball Get_outcome_clanger.1,type_Hard Ball Get_outcome_effective.1,type_Hard Ball Get_outcome_ineffective.1,type_Hard Ball Get_outcome_clanger.1,type_Spoil_outcome_effective.1,type_Spoil_outcome_ineffective.1,type_Spoil_outcome_clanger.1,type_Gather_outcome_effective.1,type_Gather_outcome_ineffective.1,type_Gather_outcome_clanger.1,type_Free For_outcome_effective.1,type_Free For_outcome_ineffective.1,type_Free For_outcome_clanger.1,type_Knock On_outcome_effective.1,type_Knock On_outcome_ineffective.1,type_Knock On_outcome_clanger.1,type_Shot_outcome_effective.1,type_Shot_outcome_ineffective.1,type_Shot_outcome_clanger.1,type_Tackle_outcome_effective.1,type_Tackle_outcome_ineffective.1,type_Tackle_outcome_clanger.1,type_Error_outcome_effective.1,type_Error_outcome_ineffective.1,type_Error_outcome_clanger.1,quarter.1,quarter_seconds.1,overall_seconds.1,start_x.1,start_y.1,end_x.1,end_y.1,dx.1,dy.1,movement.1,type_Kick.2,type_Handball.2,type_Carry.2,type_Uncontested Mark.2,type_Contested Mark.2,type_Loose Ball Get.2,type_Hard Ball Get.2,type_Spoil.2,type_Gather.2,type_Free For.2,type_Knock On.2,type_Shot.2,type_Tackle.2,type_Error.2,outcome_effective.2,outcome_ineffective.2,outcome_clanger.2,type_Kick_outcome_effective.2,type_Kick_outcome_ineffective.2,type_Kick_outcome_clanger.2,type_Handball_outcome_effective.2,type_Handball_outcome_ineffective.2,type_Handball_outcome_clang

Optuna Hyperparameter Tuning Class - HyperParameterTuner & XGBHyperparameterTuner

In [43]:
xgb_tuner = XGBHyperparameterTuner(X_train_preproc, y_train, monotonicity_constraints={})

In [44]:
xgb_tuner.tune_hyperparameters()

[I 2023-08-02 17:34:25,006] A new study created in memory with name: no-name-460944cb-77c1-4658-8af9-68d24fef1316
[I 2023-08-02 17:34:50,888] Trial 0 finished with value: 0.33581224695578904 and parameters: {'max_depth': 17, 'min_child_weight': 8, 'eta': 0.1476282549703385, 'gamma': 0.001869840680904524, 'lambda': 0.007279826439377561, 'alpha': 6.771032297853515, 'subsample': 0.49295075295558344, 'colsample_bytree': 0.5982743644667996}. Best is trial 0 with value: 0.33581224695578904.
[I 2023-08-02 17:35:06,016] Trial 1 finished with value: 0.4859418332196738 and parameters: {'max_depth': 12, 'min_child_weight': 76, 'eta': 0.04876898038218781, 'gamma': 0.15830216954708118, 'lambda': 0.24270353364586925, 'alpha': 7.438089855236683, 'subsample': 0.48212688829075323, 'colsample_bytree': 0.2212645317394681}. Best is trial 0 with value: 0.33581224695578904.
[I 2023-08-02 17:35:22,477] Trial 2 finished with value: 0.4034087492285663 and parameters: {'max_depth': 16, 'min_child_weight': 23, '

Number of finished trials:  10
Best trial:
  Value: 0.2913922002107307
  Params: 
    max_depth: 9
    min_child_weight: 65
    eta: 0.5912211573524633
    gamma: 0.04887204207320006
    lambda: 0.8499579257416128
    alpha: 0.006162970405072807
    subsample: 0.27138121618487077
    colsample_bytree: 0.5247020196937383


In [45]:
params = xgb_tuner.get_best_params()
params

{'max_depth': 9,
 'min_child_weight': 65,
 'eta': 0.5912211573524633,
 'gamma': 0.04887204207320006,
 'lambda': 0.8499579257416128,
 'alpha': 0.006162970405072807,
 'subsample': 0.27138121618487077,
 'colsample_bytree': 0.5247020196937383}

Training Model - SuperXGBClassifier class for training and predictions

In [46]:
params['objective'] = OptunaXGBParamGrid.error
params['num_rounds'] = 1000
params['early_stopping_rounds'] = 50
params['verbosity'] = 1
params['monotone_constraints'] = {}

In [47]:
super_xgb = SuperXGBClassifier(X_train = X_train_preproc, 
                               y_train = y_train, 
                               X_test = X_test_preproc, 
                               y_test = y_test,
                               X_cal = X_val_preproc,
                               y_cal = y_val,
                               params = params)

In [48]:
super_xgb.fit()

[0]	validation_0-logloss:0.41790	validation_1-logloss:0.41834
[1]	validation_0-logloss:0.34399	validation_1-logloss:0.34475
[2]	validation_0-logloss:0.31371	validation_1-logloss:0.31445
[3]	validation_0-logloss:0.30145	validation_1-logloss:0.30241
[4]	validation_0-logloss:0.29709	validation_1-logloss:0.29825
[5]	validation_0-logloss:0.29313	validation_1-logloss:0.29467
[6]	validation_0-logloss:0.29104	validation_1-logloss:0.29310
[7]	validation_0-logloss:0.29040	validation_1-logloss:0.29281
[8]	validation_0-logloss:0.28914	validation_1-logloss:0.29212
[9]	validation_0-logloss:0.28829	validation_1-logloss:0.29131
[10]	validation_0-logloss:0.28811	validation_1-logloss:0.29123
[11]	validation_0-logloss:0.28753	validation_1-logloss:0.29081
[12]	validation_0-logloss:0.28667	validation_1-logloss:0.29027
[13]	validation_0-logloss:0.28634	validation_1-logloss:0.29013
[14]	validation_0-logloss:0.28585	validation_1-logloss:0.28998
[15]	validation_0-logloss:0.28575	validation_1-logloss:0.29007
[1

In [49]:
super_xgb.xgb_model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.5247020196937383, early_stopping_rounds=50,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.04887204207320006, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.5912211573524633, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=9, max_leaves=None,
              min_child_weight=65, missing=nan, monotone_constraints={},
              n_estimators=1000, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [50]:
super_xgb.xgb_model.get_booster().feature_names

['type_Kick',
 'type_Handball',
 'type_Carry',
 'type_Uncontested Mark',
 'type_Contested Mark',
 'type_Loose Ball Get',
 'type_Hard Ball Get',
 'type_Spoil',
 'type_Gather',
 'type_Free For',
 'type_Knock On',
 'type_Shot',
 'type_Tackle',
 'type_Error',
 'outcome_effective',
 'outcome_ineffective',
 'outcome_clanger',
 'type_Kick_outcome_effective',
 'type_Kick_outcome_ineffective',
 'type_Kick_outcome_clanger',
 'type_Handball_outcome_effective',
 'type_Handball_outcome_ineffective',
 'type_Handball_outcome_clanger',
 'type_Carry_outcome_effective',
 'type_Carry_outcome_ineffective',
 'type_Carry_outcome_clanger',
 'type_Uncontested Mark_outcome_effective',
 'type_Uncontested Mark_outcome_ineffective',
 'type_Uncontested Mark_outcome_clanger',
 'type_Contested Mark_outcome_effective',
 'type_Contested Mark_outcome_ineffective',
 'type_Contested Mark_outcome_clanger',
 'type_Loose Ball Get_outcome_effective',
 'type_Loose Ball Get_outcome_ineffective',
 'type_Loose Ball Get_outcome_c

In [52]:
train_preds = super_xgb.predict(X_train_preproc)
test_preds = super_xgb.predict(X_test_preproc)

In [55]:
train_probas = super_xgb.predict_proba(X_train_preproc)[:, 1]
test_probas = super_xgb.predict_proba(X_test_preproc)[:, 1]
val_probas = super_xgb.predict_proba(X_val_preproc)[:, 1]

In [56]:
super_xgb.calibrate()

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [57]:
train_cal_probas = super_xgb.predict_proba(X_train_preproc, calibrate=True)
test_cal_probas = super_xgb.predict_proba(X_test_preproc, calibrate=True)

Check Average Predictions

In [59]:
train_probas.mean(), y_train.mean(), train_cal_probas.mean()

(0.11643901, 0.11534220065375746, 0.11542815383268523)

In [60]:
test_probas.mean(), y_test.mean(), test_cal_probas.mean()

(0.11648989, 0.11534038759771703, 0.11546524734489899)

Export model

In [61]:
super_xgb.export_model(model_output_path + "/" + model_file_name + ".joblib")

Export data and predictions

In [64]:
train_export = pd.concat([y_train, X_train], axis=1)
train_export['xgb_preds'] = train_preds
train_export['xgb_probas'] = train_probas
train_export['xgb_probas_cal'] = train_cal_probas
train_export.to_csv(prediction_output_path + '/train_predictions_' + model_file_name + '.csv', index = False)
train_export.head()

,scores,action_type,type_Kick,type_Handball,type_Carry,type_Uncontested Mark,type_Contested Mark,type_Loose Ball Get,type_Hard Ball Get,type_Spoil,type_Gather,type_Free For,type_Knock On,type_Shot,type_Tackle,type_Error,outcome_type,outcome_effective,outcome_ineffective,outcome_clanger,type_Kick_outcome_effective,type_Kick_outcome_ineffective,type_Kick_outcome_clanger,type_Handball_outcome_effective,type_Handball_outcome_ineffective,type_Handball_outcome_clanger,type_Carry_outcome_effective,type_Carry_outcome_ineffective,type_Carry_outcome_clanger,type_Uncontested Mark_outcome_effective,type_Uncontested Mark_outcome_ineffective,type_Uncontested Mark_outcome_clanger,type_Contested Mark_outcome_effective,type_Contested Mark_outcome_ineffective,type_Contested Mark_outcome_clanger,type_Loose Ball Get_outcome_effective,type_Loose Ball Get_outcome_ineffective,type_Loose Ball Get_outcome_clanger,type_Hard Ball Get_outcome_effective,type_Hard Ball Get_outcome_ineffective,type_Hard Ball Get_outcome_clanger,type_Spoil_outcome_effective,type_Spoil_outcome_ineffective,type_Spoil_outcome_clanger,type_Gather_outcome_effective,type_Gather_outcome_ineffective,type_Gather_outcome_clanger,type_Free For_outcome_effective,type_Free For_outcome_ineffective,type_Free For_outcome_clanger,type_Knock On_outcome_effective,type_Knock On_outcome_ineffective,type_Knock On_outcome_clanger,type_Shot_outcome_effective,type_Shot_outcome_ineffective,type_Shot_outcome_clanger,type_Tackle_outcome_effective,type_Tackle_outcome_ineffective,type_Tackle_outcome_clanger,type_Error_outcome_effective,type_Error_outcome_ineffective,type_Error_outcome_clanger,quarter,quarter_seconds,overall_seconds,start_x,start_y,end_x,end_y,dx,dy,movement,action_type.1,type_Kick.1,type_Handball.1,type_Carry.1,type_Uncontested Mark.1,type_Contested Mark.1,type_Loose Ball Get.1,type_Hard Ball Get.1,type_Spoil.1,type_Gather.1,type_Free For.1,type_Knock On.1,type_Shot.1,type_Tackle.1,type_Error.1,outcome_type.1,outcome_effective.1,outcome_ineffective.1,outcome_clanger.1,type_Kick_outcome_effective.1,type_Kick_outcome_ineffective.1,type_Kick_outcome_clanger.1,type_Handball_outcome_effective.1,type_Handball_outcome_ineffective.1,type_Handball_outcome_clanger.1,type_Carry_outcome_effective.1,type_Carry_outcome_ineffective.1,type_Carry_outcome_clanger.1,type_Uncontested Mark_outcome_effective.1,type_Uncontested Mark_outcome_ineffective.1,type_Uncontested Mark_outcome_clanger.1,type_Contested Mark_outcome_effective.1,type_Contested Mark_outcome_ineffective.1,type_Contested Mark_outcome_clanger.1,type_Loose Ball Get_outcome_effective.1,type_Loose Ball Get_outcome_ineffective.1,type_Loose Ball Get_outcome_clanger.1,type_Hard Ball Get_outcome_effective.1,type_Hard Ball Get_outcome_ineffective.1,type_Hard Ball Get_outcome_clanger.1,type_Spoil_outcome_effective.1,type_Spoil_outcome_ineffective.1,type_Spoil_outcome_clanger.1,type_Gather_outcome_effective.1,type_Gather_outcome_ineffective.1,type_Gather_outcome_clanger.1,type_Free For_outcome_effective.1,type_Free For_outcome_ineffective.1,type_Free For_outcome_clanger.1,type_Knock On_outcome_effective.1,type_Knock On_outcome_ineffective.1,type_Knock On_outcome_clanger.1,type_Shot_outcome_effective.1,type_Shot_outcome_ineffective.1,type_Shot_outcome_clanger.1,type_Tackle_outcome_effective.1,type_Tackle_outcome_ineffective.1,type_Tackle_outcome_clanger.1,type_Error_outcome_effective.1,type_Error_outcome_ineffective.1,type_Error_outcome_clanger.1,quarter.1,quarter_seconds.1,overall_seconds.1,start_x.1,start_y.1,end_x.1,end_y.1,dx.1,dy.1,movement.1,action_type.2,type_Kick.2,type_Handball.2,type_Carry.2,type_Uncontested Mark.2,type_Contested Mark.2,type_Loose Ball Get.2,type_Hard Ball Get.2,type_Spoil.2,type_Gather.2,type_Free For.2,type_Knock On.2,type_Shot.2,type_Tackle.2,type_Error.2,outcome_type.2,outcome_effective.2,outcome_ineffective.2,outcome_clanger.2,type_Kick_outcome_effective.2,type_Kick_outcome_ineffective.2,type_Kick_outcome_clanger.2,type_Ha

In [65]:
test_export = pd.concat([X_test, y_test], axis=1)
test_export['xgb_preds'] = test_preds
test_export['xgb_probas'] = test_probas
test_export['xgb_probas_cal'] = test_cal_probas
test_export.to_csv(prediction_output_path + '/test_predictions_' + model_file_name + '.csv', index = False)
test_export.head()

,action_type,type_Kick,type_Handball,type_Carry,type_Uncontested Mark,type_Contested Mark,type_Loose Ball Get,type_Hard Ball Get,type_Spoil,type_Gather,type_Free For,type_Knock On,type_Shot,type_Tackle,type_Error,outcome_type,outcome_effective,outcome_ineffective,outcome_clanger,type_Kick_outcome_effective,type_Kick_outcome_ineffective,type_Kick_outcome_clanger,type_Handball_outcome_effective,type_Handball_outcome_ineffective,type_Handball_outcome_clanger,type_Carry_outcome_effective,type_Carry_outcome_ineffective,type_Carry_outcome_clanger,type_Uncontested Mark_outcome_effective,type_Uncontested Mark_outcome_ineffective,type_Uncontested Mark_outcome_clanger,type_Contested Mark_outcome_effective,type_Contested Mark_outcome_ineffective,type_Contested Mark_outcome_clanger,type_Loose Ball Get_outcome_effective,type_Loose Ball Get_outcome_ineffective,type_Loose Ball Get_outcome_clanger,type_Hard Ball Get_outcome_effective,type_Hard Ball Get_outcome_ineffective,type_Hard Ball Get_outcome_clanger,type_Spoil_outcome_effective,type_Spoil_outcome_ineffective,type_Spoil_outcome_clanger,type_Gather_outcome_effective,type_Gather_outcome_ineffective,type_Gather_outcome_clanger,type_Free For_outcome_effective,type_Free For_outcome_ineffective,type_Free For_outcome_clanger,type_Knock On_outcome_effective,type_Knock On_outcome_ineffective,type_Knock On_outcome_clanger,type_Shot_outcome_effective,type_Shot_outcome_ineffective,type_Shot_outcome_clanger,type_Tackle_outcome_effective,type_Tackle_outcome_ineffective,type_Tackle_outcome_clanger,type_Error_outcome_effective,type_Error_outcome_ineffective,type_Error_outcome_clanger,quarter,quarter_seconds,overall_seconds,start_x,start_y,end_x,end_y,dx,dy,movement,action_type.1,type_Kick.1,type_Handball.1,type_Carry.1,type_Uncontested Mark.1,type_Contested Mark.1,type_Loose Ball Get.1,type_Hard Ball Get.1,type_Spoil.1,type_Gather.1,type_Free For.1,type_Knock On.1,type_Shot.1,type_Tackle.1,type_Error.1,outcome_type.1,outcome_effective.1,outcome_ineffective.1,outcome_clanger.1,type_Kick_outcome_effective.1,type_Kick_outcome_ineffective.1,type_Kick_outcome_clanger.1,type_Handball_outcome_effective.1,type_Handball_outcome_ineffective.1,type_Handball_outcome_clanger.1,type_Carry_outcome_effective.1,type_Carry_outcome_ineffective.1,type_Carry_outcome_clanger.1,type_Uncontested Mark_outcome_effective.1,type_Uncontested Mark_outcome_ineffective.1,type_Uncontested Mark_outcome_clanger.1,type_Contested Mark_outcome_effective.1,type_Contested Mark_outcome_ineffective.1,type_Contested Mark_outcome_clanger.1,type_Loose Ball Get_outcome_effective.1,type_Loose Ball Get_outcome_ineffective.1,type_Loose Ball Get_outcome_clanger.1,type_Hard Ball Get_outcome_effective.1,type_Hard Ball Get_outcome_ineffective.1,type_Hard Ball Get_outcome_clanger.1,type_Spoil_outcome_effective.1,type_Spoil_outcome_ineffective.1,type_Spoil_outcome_clanger.1,type_Gather_outcome_effective.1,type_Gather_outcome_ineffective.1,type_Gather_outcome_clanger.1,type_Free For_outcome_effective.1,type_Free For_outcome_ineffective.1,type_Free For_outcome_clanger.1,type_Knock On_outcome_effective.1,type_Knock On_outcome_ineffective.1,type_Knock On_outcome_clanger.1,type_Shot_outcome_effective.1,type_Shot_outcome_ineffective.1,type_Shot_outcome_clanger.1,type_Tackle_outcome_effective.1,type_Tackle_outcome_ineffective.1,type_Tackle_outcome_clanger.1,type_Error_outcome_effective.1,type_Error_outcome_ineffective.1,type_Error_outcome_clanger.1,quarter.1,quarter_seconds.1,overall_seconds.1,start_x.1,start_y.1,end_x.1,end_y.1,dx.1,dy.1,movement.1,action_type.2,type_Kick.2,type_Handball.2,type_Carry.2,type_Uncontested Mark.2,type_Contested Mark.2,type_Loose Ball Get.2,type_Hard Ball Get.2,type_Spoil.2,type_Gather.2,type_Free For.2,type_Knock On.2,type_Shot.2,type_Tackle.2,type_Error.2,outcome_type.2,outcome_effective.2,outcome_ineffective.2,outcome_clanger.2,type_Kick_outcome_effective.2,type_Kick_outcome_ineffective.2,type_Kick_outcome_clanger.2,type_Handball_

Save preprocessor

In [ ]:
# joblib.dump(preprocessor, preprocessor_output_path + preprocessor_file_name + ".joblib")